A Diffusion "Experiment"
==

This notebook is a simulation of
random particle motion, and how they related
to a diffusion coefficient.

In [226]:
# Load code related to plotting:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh import palettes
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource
output_notebook()

# Load code to make our simulation interactive in this notebook.
import ipywidgets as widgets

# Our simulation will use functions from numpy
import numpy as np

Loading BokehJS ...

In [308]:
# The simulation:

# Parameters for the experiment:
N=4000 # number of particles
L=0.05 # length scale for random steps
dt=1.0 # time scale for random steps
box_dx=1.0 # dimensions of the box the particles move around in.
box_dy=1.0 # 
box_dz=1.0 #
# We will look at the flux through a surface in the middle of
# the box, perpendicular to the x axis. 
box_A=box_dy*box_dz # area of that surface
box_V=box_dx*box_dy*box_dz # volume of the whole box.

In [335]:
# The state of our simulation, an instantaneous description
# of the evolving system:
def initialize():
    global part_x,part_y,part_z,part_mass
    global t, mass_on_left, times, gradients
    
    part_x=np.random.uniform(low=-box_dx/2,high=box_dx/2,size=N)
    part_y=np.random.uniform(low=-box_dy/2,high=box_dy/2,size=N)
    part_z=np.random.uniform(low=-box_dz/2,high=box_dz/2,size=N)

    # particle mass is initialize with the x coordinate to create a 
    # smooth gradient in x.
    part_mass=1+part_x # to be concrete, let's call the units grams.

    t=0.0 # simulation time.

    # Simulation will update particle locations, and at each step
    # calculate these summary quantities:
    mass_on_left=[] # mass of particles on the left half of the box
    times=[]        # simulation time associated with each step.
    gradients=[]    # mass gradient 

# The code for our simulation has two tasks:
#  1. Update the state (part_x,part_y,part_z,t)
#  2. Record summary information.

def step_forward():
    # Make these global so they can be updated from within
    # this function.
    global part_x,part_y,part_z,t
    
    # For each direction, add a random jump, uniformly
    # random between [-L,+L]
    part_x += np.random.uniform(low=-L,high=L,size=N)
    part_y += np.random.uniform(low=-L,high=L,size=N)
    part_z += np.random.uniform(low=-L,high=L,size=N)

    # Particles that would leave our box get "bounced"
    # back into the box
    x_outside=part_x - part_x.clip(-box_dx/2,box_dx/2)
    y_outside=part_y - part_y.clip(-box_dy/2,box_dy/2)
    z_outside=part_z - part_z.clip(-box_dz/2,box_dz/2)
    part_x-=2*x_outside
    part_y-=2*y_outside
    part_z-=2*z_outside
    t+=dt
    
def record():
    times.append(t)
    mass_on_left.append( part_mass[part_x<0.0].sum() )
    slope,inter =np.polyfit(part_x,part_mass,1)
    gradients.append(slope*N/box_V)

initialize() # Create initial state
record() # Save the initial state.

In [336]:
part_data=ColumnDataSource(dict(x=part_x,y=part_y,m=part_mass))
colors=linear_cmap(field_name='m',low=part_mass.min(),high=part_mass.max(),
                   palette=palettes.Plasma256)

p = figure(x_range=[-box_dx/2, box_dx/2], y_range=[-box_dy/2, box_dy/2],
           plot_width=800, plot_height=300)

r = p.circle(x='x',y='y', radius=0.005, 
             fill_color=colors, fill_alpha=0.6, line_color=None, source=part_data)
color_bar = ColorBar(color_mapper=colors['transform'], width=8,  location=(0,0))

p.add_layout(color_bar, 'right')

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

def plot_update(step,length):
    global L
    L=length
    if (step==0) and (t>0):
        initialize()
        r.data_source.data['m']=part_mass
    elif step>0:
        step_forward()
    record()

    r.data_source.data['x']=part_x
    r.data_source.data['y']=part_y
    push_notebook(handle=target)
        
interact(plot_update,
         step=widgets.Play(value=0,
                           min=0, max=10000,
                           step=1, interval=20),
         length=widgets.FloatSlider(value=0.05,min=0,max=0.2,
                                step=0.005,
                                description='L:',
                                orientation='horizontal',
                                readout=True,
                                readout_format='.3f'))
;

interactive(children=(Play(value=0, description='step', interval=20, max=10000), FloatSlider(value=0.05, descr…

''

How did the particle distribution evolve over time?
--

In [344]:
# We record mass on the left side. 
p2 = figure(plot_width=800, plot_height=300)
p2.line(times,mass_on_left)
p2.xaxis.axis_label='Time'
p2.yaxis.axis_label='Mass on left side'
show(p2)

Relate mass on the left side to the flux through the center

In [345]:
# mass flux through x=0, from left to right, is equal to 
# th negative of the the change
# in mass on the left side (since the rest of the box is closed)
dMdt=(mass_on_left[-1] - mass_on_left[0])/(times[-1] - times[0])
flux=-dMdt/box_A # mass/area*time
print(f"Average flux so far: {flux:.4f} grams/m^3") # UNITS!

Average flux so far: -0.3356 grams/m^3


In [346]:
# The gradient is changing over time, so take its average, too.
gradient=np.mean(gradients)
print(f"Average gradient so far: {gradient:.2f} weird units")

Average gradient so far: 3837.56 weird units


In [347]:
# What kind of diffusion coefficient do we "observe" ?
# flux ~ -D dC/dx
Dobs=-flux / gradient
print(f"Observed diffusion coefficient: {Dobs:.6f} m^2/s")

Observed diffusion coefficient: 0.00009 m^2/s


In [348]:
u=L/dt

Dscaled=u*L
print(f"Estimated diffusion coefficent from scaling: {Dscaled:.6f} m^2/s")

Estimated diffusion coefficent from scaling: 0.00040 m^2/s


Good news: The sign is correct.

Better news: The order of magnitude is roughly correct. 

For scaling relationships and the
types of approximations we will encounter, that is often as good as it gets.

If you are curious
===

Why are we off by almost an order of magnitude?


Fokker-Planck Solution for Random Walks
--

Uniform $D$, $R$ is uniformly distributed $[-1,1]$

$$ \Delta x = R  \sqrt{2 r^{-1} D \Delta t} $$ 

where $r$ is the standard deviation of $R$, $r=\sqrt{E[R^2]}=1/3$.

In my toy example $\Delta x$ is uniform over $[-L,L]$

In relation to Fokker-Planck, I'm assuming $\sqrt{2r^{-1}D\Delta t}=L$.

That means the $D$ that I'm *actually* simulating is...

$$D = \frac{r}{2} \frac{L^2}{\Delta t} = \frac{D_{scaled}}{6}$$



In [349]:
Dapp=u*L/6
print(f"Observed D:    {Dobs:.6f} m^2/s")
print(f"Scaled D:      {Dscaled:.6f} m^2/s")
print(f"Appoximated D: {Dapp:.6f} m^2/s")


Observed D:    0.00009 m^2/s
Scaled D:      0.00040 m^2/s
Appoximated D: 0.00007 m^2/s


Still not exact??

Unsteadiness and randomness affect the "observed" diffusion.  The gradient is changing in time, and it can take very large numbers of particles to even out the noise of a random process. As the number of particles is increased and the walls of the box moved away from our area of interest, the observed and expected diffusion coefficients will converge.